In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.animation as animation

from torch.utils.data import Dataset
# import binvox
import os
import json
import trimesh
import skimage
import random
# from mesh_to_sdf import mesh_to_voxels, sample_sdf_near_surface, scale_to_unit_sphere
import mesh_to_sdf
import pyrender
import numpy as np
import matplotlib.pyplot as plt
import io
from deep_sdf import utils, metrics, plotting
import math
import pandas as pd
import pytorch3d
import torch

import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

/home/wulff/anaconda3/envs/deep3dcomp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
i/n

0.1

In [2]:

folder = "/mnt/hdd/ShapeNetCore.v2/02691156" # planes
folder = "/mnt/hdd/ShapeNetCore.v2/04256520"    # sofas
# folder = "/mnt/hdd/ShapeNetCore.v2/03636649"    # lamps



i = 0
n = 0
for fold in os.listdir(folder):
    p = os.path.join(folder, fold, "models/model_normalized.obj")
    mesh = trimesh.load(p)
    n += 1
    try:
        ret = mesh_to_sdf.mesh_to_voxels(mesh, voxel_resolution=128, check_result=True, pad=True, sign_method="depth")
        print("Success", p)
        i += 1
    except mesh_to_sdf.BadMeshException:
        pass
        # print("BadMeshException")

print("Total", i, " out of ", len(os.listdir(folder)))

# idxs_neg = np.argwhere(sdf < 1).reshape(-1)
# # idxs_neg = np.argwhere((-0.1 < sdf) & (sdf < 0.0)).reshape(-1)

# print(len(idxs_neg))

# anim = plotting.render_sdf_vid(points[idxs_neg], sdf[idxs_neg])
# plt.close()

# from IPython.display import HTML
# HTML(anim.to_jshtml())

KeyboardInterrupt: 

In [17]:
0.001788/0.001

1.788

In [ ]:
N = 144 # CD: 0.0006966032096036104
N = 146 # CD: 0.0007368986723927739
N = 150 # CD: 0.008003759857797957
N = 155 # CD: 0.0015067354416739066


voxel_size = 2.0 / (N - 1)

# p = '/mnt/hdd/ShapeNetCore.v2/04256520/48e31e3a3e2aa5c0cabc196a7f35f1aa/models/model_normalized.obj'
# p = '/mnt/hdd/ShapeNetCore.v2/04256520/104256e5bb73b0b719fb4103277a6b93/models/model_normalized.obj'
p = "/mnt/hdd/ShapeNetCore.v2/02691156/d0456644386d9149ce593c35f70d3f/models/model_normalized.obj"
p = "../../shared/deepsdfcomp/data/manifold_meshes/02691156/d0456644386d9149ce593c35f70d3f.obj"

mesh = utils.scale_to_unit_sphere(utils.as_mesh(trimesh.load(p)))
voxels = mesh_to_sdf.mesh_to_voxels(mesh, voxel_resolution=N, check_result=True, pad=True, sign_method="depth")

verts, faces, normals, values = skimage.measure.marching_cubes(voxels, level=0.0, spacing=[voxel_size] * 3, method="lewiner")

recon = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)
recon = utils.scale_to_unit_sphere(recon)
print("CD:", metrics.compute_metric(mesh, recon)[0])
recon.show()


In [11]:
print("CD:", metrics.compute_metric(mesh, utils.scale_to_unit_sphere(recon))[0])

CD: 0.0007127533561920704


In [ ]:
p = '/mnt/hdd/ShapeNetCore.v2/04256520/103b76b2594a1582eaf14273fa406ffc/models/model_normalized.obj'
p = "../../shared/deepsdfcomp/data/manifold_meshes/02691156/d068bfa97f8407e423fc69eefd95e6d3.obj"
p = "../../shared/deepsdfcomp/data/manifold_meshes/02691156/d0456644386d9149ce593c35f70d3f.obj"
trimesh.load(p).show()

In [ ]:
N = 400
voxel_size = 2.0 / (N - 1)

p = '/mnt/hdd/ShapeNetCore.v2/02691156/1bcbb0267f5f1d53c6c0edf9d2d89150/models/model_normalized.obj'
mesh = trimesh.load(p)
voxels = mesh_to_sdf.mesh_to_voxels(mesh, voxel_resolution=N, check_result=False, pad=False, sign_method="depth")
print("Got voxels")
verts, faces, normals, values = skimage.measure.marching_cubes(voxels, level=0.0, spacing=[voxel_size] * 3, method="lewiner")

recon = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)
recon = utils.scale_to_unit_sphere(recon)
print("CD:", metrics.compute_metric(utils.scale_to_unit_sphere(mesh), recon)[0])
recon.show()


In [26]:
import pandas as pd
import ast
import matplotlib.pyplot as plt

# df = pd.read_csv("examples/planes/Evaluation/2000/chamfer.csv", delimiter=";")
df_line = pd.read_csv("../../shared/deepsdfcomp/searches/double_nonlinearity/line/Evaluation/2000/chamfer.csv", delimiter=";")
df_plane = pd.read_csv("../../shared/deepsdfcomp/searches/double_nonlinearity/plane/Evaluation/2000/chamfer.csv", delimiter=";")
df_baseline = pd.read_csv("../../shared/deepsdfcomp/searches/double_nonlinearity/baseline/Evaluation/2000/chamfer.csv", delimiter=";")
df_siren = pd.read_csv("../../shared/deepsdfcomp/searches/double_nonlinearity/siren_new_schedule/Evaluation/2000/chamfer.csv", delimiter=";")

# fig, ax = plt.subplots(1, 3, figsize=(15, 5))
# titles = ["NL Combination: Line", "SIREN", "Baseline"]
# for i, df in enumerate([df1, df2, df_baseline]):
#     df.head()
#     print(df.mean())
#     print(df.median())
#     df.hist("chamfer_dist", bins=100, ax=ax[i])
#     ax[i].set_xlim((0, 0.010))
#     # ax[i].set_ylim((0, 350))
#     ax[i].set_title(titles[i])

In [28]:
df_plane.mean()

/tmp/ipykernel_2346647/800213934.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_plane.mean()


chamfer_dist          0.000227
normal_consistency    0.018330
dtype: float64

In [16]:
df_siren.sort_values("chamfer_dist", ascending=False).drop(columns=["all_chamfer_dist"])

,shape,chamfer_dist,normal_consistency
412,ShapeNetV2/02691156/fc16704314dc71ee9a6e43b878d5b335,8.756794e-03,0.012752
360,ShapeNetV2/02691156/f6e6fd724a9eab24ba8e93696257b3fc,5.344461e-03,0.016526
438,ShapeNetV2/02691156/fef1c15a5db21b00a67cc8f661e7890a,4.676031e-03,0.015771
153,ShapeNetV2/02691156/df25be12ae47d2517ef7776b3bf5815c,2.579081e-03,0.017206
390,ShapeNetV2/02691156/f9db62e6a88f0d7129343faf3bbffb15,2.484231e-03,0.010837
...,...,...,...
244,ShapeNetV2/02691156/e87902d01d40c71521b1ceea9c93aa79,1.015518e-05,0.017794
16,ShapeNetV2/02691156/d28a3cd771b2bf1e71d03b466c72ce41,9.532152e-06,0.015387
152,ShapeNetV2/02691156/df1c68e94259c6cf5c7319b71bdce6e,7.015172e-06,0.020189
154,ShapeNetV2/02691156/df411aa240fe48d5855eb7906a7a7a04,2.100116e-06,0.016054


In [ ]:
trimesh.load("/mnt/hdd/ShapeNetCore.v2/02691156/fef1c15a5db21b00a67cc8f661e7890a/models/model_normalized.obj").show()

In [56]:
[torch.Tensor(_) for _ in mesh.faces][0].shape

torch.Size([3])

In [4]:
from pytorch3d.structures import Meshes
from pytorch3d import structures, io, loss

filename = "/mnt/hdd/ShapeNetCore.v2/02691156/fef1c15a5db21b00a67cc8f661e7890a/models/model_normalized.obj"
mesh = utils.as_mesh(trimesh.load(filename))

# verts, faces = io.load_ply(filename)
# verts, faces, aux = io.load_obj(filename)
# meshes = Meshes(verts=[verts], faces=[faces.verts_idx])

meshes = trimesh_to_pytorch3d_meshes(mesh)
loss.mesh_normal_consistency(meshes)

tensor(0.7256)

In [13]:
filename = "../../shared/deepsdfcomp/searches/double_nonlinearity/baseline/Reconstructions/2000/Meshes/ShapeNetV2/02691156/d0456644386d9149ce593c35f70d3f.ply"
mesh = utils.as_mesh(trimesh.load(filename))
meshes = utils.trimesh_to_pytorch3d_meshes([mesh])
loss.mesh_normal_consistency(meshes)

tensor(0.0228)

In [11]:
meshes.faces_list()

[tensor([[    2,     1,     0],
         [    4,     3,     0],
         [    4,     0,     1],
         ...,
         [45684, 45682, 45683],
         [45684, 45683, 45685],
         [45685, 45683, 45680]]),
 tensor([[    2,     1,     0],
         [    4,     3,     0],
         [    4,     0,     1],
         ...,
         [45684, 45682, 45683],
         [45684, 45683, 45685],
         [45685, 45683, 45680]])]

In [64]:
filename = "../../shared/deepsdfcomp/searches/double_nonlinearity/siren_new_schedule/Reconstructions/2000/Meshes/ShapeNetV2/02691156/d0456644386d9149ce593c35f70d3f.ply"
mesh = utils.as_mesh(trimesh.load(filename))
meshes = trimesh_to_pytorch3d_meshes(mesh)
loss.mesh_normal_consistency(meshes)

tensor(0.0197)

In [21]:
torch.Tensor(np.array(mesh.faces)).size(2)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [ ]:
experiment_dirs = [
    "../../shared/deepsdfcomp/searches/double_nonlinearity/baseline",
    # "../../shared/deepsdfcomp/searches/double_nonlinearity/all_latentsize=200_width=256_lr=5e-4_int=150",
    "../../shared/deepsdfcomp/searches/double_nonlinearity/line",
    "../../shared/deepsdfcomp/searches/double_nonlinearity/plane",
    "../../shared/deepsdfcomp/searches/double_nonlinearity/siren_new_schedule",
]

shape_ids = [
    "fc16704314dc71ee9a6e43b878d5b335",
    "f6e6fd724a9eab24ba8e93696257b3fc",  
    "fef1c15a5db21b00a67cc8f661e7890a",
    "df25be12ae47d2517ef7776b3bf5815c",
    "f9db62e6a88f0d7129343faf3bbffb15",
]

plotting.plot_reconstruction_comparison(experiment_dirs, shape_ids)